In [1]:
# Import required libraries
import pandas as pd
from datetime import date
import time
import smtplib
import requests
from bs4 import BeautifulSoup

In [2]:
# Define URL and headers
url = 'https://www.amazon.com/Nice-Story-Now-Show-Data/dp/B09XZMCJZR/ref=sr_1_4?crid=VEQSK2K3DQS6&keywords=funny+data&qid=1678664591&sprefix=funny+data%2Caps%2C166&sr=8-4'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
} # get user-agent information from: https://httpbin.org/get

# Connect to the website and pull in data
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
title = soup.find(id='productTitle').get_text().strip()
price = soup.find(id="price").get_text().strip()[1:]

print(title)
print(price)

Nice Story Now Show Me The Data: Funny Data Analyst Gift, Blank Lined Notebook for Data Analyst, Data Scientist Gift
6.99


In [3]:
import openpyxl

# Create a new Excel workbook and select the active worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active

# Define the headers and data to be written to the worksheet
header = ['Title', 'Price', 'Date']
data = [title, price, date.today()]

# Write the headers and data to the worksheet
worksheet.append(header)
worksheet.append(data)

# Save the workbook to an Excel file
workbook.save('FunnyNotebookDataset.xlsx')

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('FunnyNotebookDataset.xlsx')

# Print the data
print(df.head())

                                               Title  Price       Date
0  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13


In [4]:
# Append data to the Excel file
workbook = openpyxl.load_workbook('FunnyNotebookDataset.xlsx')
worksheet = workbook.active
worksheet.append(data)
workbook.save('FunnyNotebookDataset.xlsx')

In [5]:
# Define function to scrape the price and title from the Amazon product page
def check_price():
    # Define URL and headers
    url = 'https://www.amazon.com/Nice-Story-Now-Show-Data/dp/B09XZMCJZR/ref=sr_1_4?crid=VEQSK2K3DQS6&keywords=funny+data&qid=1678664591&sprefix=funny+data%2Caps%2C166&sr=8-4'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    # Connect to the website and pull in data
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.find(id='productTitle').get_text().strip()
    price = float(soup.find(id="price").get_text().strip()[1:])

    # Load the existing workbook if it exists, otherwise create a new one
    try:
        workbook = openpyxl.load_workbook('FunnyNotebookDataset.xlsx')
    except FileNotFoundError:
        workbook = openpyxl.Workbook()
        worksheet = workbook.active
        worksheet.append(['Title', 'Price', 'Date'])  # include headers only once

    # Select the active worksheet
    worksheet = workbook.active

    # Define the data to be written to the worksheet
    data = [title, price, date.today()]

    # Write the data to the worksheet
    worksheet.append(data)

    # Save the workbook to an Excel file
    workbook.save('FunnyNotebookDataset.xlsx')
    
    # Check if the price is below $10 and send an email if it is
    if price < 10:
        send_mail(title, url)
        
# Define function to send an email
def send_mail(title, url):
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('alknleyla@gmail.com','my_app_password') #an App Password is a 16-digit passcode that gives a less secure app or device permission to access your Google Account.
    
    subject = "The notebook you want is below $10! Now is your chance to buy!"
    body = f"Hey, this is the moment you have been waiting for. Now is your chance to pick up the notebook of your dreams ({title}). Don't mess it up! Link here: {url}"

    msg = f"Subject: {subject}\n\n{body}"

    server.sendmail(
        'alknleyla@gmail.com',
        ['alknleyla@gmail.com'],
        #['recipient1@example.com', 'recipient2@example.com'],
        msg
    )

    server.quit()


In [ ]:
# Run the check_price() function for 5 seconds for demonstration, then stop it manually
for i in range(5):
    check_price()
    time.sleep(1)

In [7]:
# Read the results
df = pd.read_excel('FunnyNotebookDataset.xlsx')
print(df.head())

                                               Title  Price       Date
0  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13
1  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13
2  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13
3  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13
4  Nice Story Now Show Me The Data: Funny Data An...   6.99 2023-03-13


In [ ]:
# This runs the check_price() function every 24 hours
while True:
    check_price()
    time.sleep(86400)